In [6]:
from mrequests import MattermostRequests

In [7]:
# Mattermost credentials should to be placed outside of this repo.
mr = MattermostRequests('../mattermost_credentials.json', 'mattermost-openapi-v4.yaml')

#### API testing

A motor feladata, hogy végrehajt egy teszt sort, és értékeli az eredményt. Addig folyik a tesztelés, amíg egy hibás válasz nem érkezik. (Hiba az is, ha nem érkezik válasz egy megadott időn belül, de itt figyelembe kell venni a hálózati késleltetést is.) A fuzzingot ezen a ponton úgy kéne megvalósítani, hogy az előbbi fuzzoló metódust használva a teszt alatt lévő API metódus össze lehetséges paraméterét végigtesztelni az összes lehetséges kombinációban. (Ez elvben nagyon sok, úgyhogy inkább az összes lehetséges teszt egy random részhalmazával érdemes próbálkozni.) A visszajövő válaszokat pedig minden esetben a séma alapján validálni kell (lásd 2. feladat).

In [8]:
import datetime

In [9]:
team_id = mr.get_teams() #[0][id]
channel_id = mr.get_public_channels(team_id=team_id) #[0][id]
apitestlist = []

In [11]:
def test_method_a(szam):
    if szam == 2:
        return 'jegesmedve'

def test_method_b(noveny, allat):
    if noveny == 'fenyo':
        return {
            'noveny': 'ez egy ' + noveny,
            'allat': 'ez egy ' + allat,
            'szin': 'zold',
        }

def test_method_c(szin):
    return szin.upper()    

test_flow = [
    { 'input_data': { 'szam': 2 }, 'target': test_method_a, 'post_process': lambda x: { 'allat': x } },
    { 'input_data': { 'noveny': 'fenyo' }, 'target': test_method_b, 'post_process': lambda x: { 'szin': x['szin'] } },
    { 'input_data': None, 'target': test_method_c, 'post_process': None}
]

In [12]:
def run_test(flow):
    result_data = None
    for i, action in enumerate(flow):
        print('Test ' + str(i + 1) + '/' + str(len(flow)))
        
        # prepare variables
        input_data = action['input_data'] or {}
        input_data.update(result_data or {})
        target = action['target']
        post_process = action['post_process']
        
        # fuzzing input
        # ....
        print('Input: ' + str(input_data))
        
        # run target
        result = target(**input_data)
        print('Output: ' + str(result))
        
        # validate result
        # ....
        
        # prepare result for next 
        result_data = post_process(result) if post_process is not None else result
        
        # save
        dict={'Input':input_data, 'output':result, 'teamID':team_id, 'channelID':channel_id}
        apitestlist.append(dict.copy())
run_test(test_flow)


Test 1/3
Input: {'szam': 2}
Output: jegesmedve
Test 2/3
Input: {'noveny': 'fenyo', 'allat': 'jegesmedve'}
Output: {'noveny': 'ez egy fenyo', 'allat': 'ez egy jegesmedve', 'szin': 'zold'}
Test 3/3
Input: {'szin': 'zold'}
Output: ZOLD


[{'Input': {'szam': 2},
  'output': 'jegesmedve',
  'teamID': [],
  'channelID': {'id': 'api.context.404.app_error',
   'message': 'Sorry, we could not find the page.',
   'detailed_error': "There doesn't appear to be an api call for the url='/api/v4/teams/[]/channels'.",
   'status_code': 404}},
 {'Input': {'noveny': 'fenyo', 'allat': 'jegesmedve'},
  'output': {'noveny': 'ez egy fenyo',
   'allat': 'ez egy jegesmedve',
   'szin': 'zold'},
  'teamID': [],
  'channelID': {'id': 'api.context.404.app_error',
   'message': 'Sorry, we could not find the page.',
   'detailed_error': "There doesn't appear to be an api call for the url='/api/v4/teams/[]/channels'.",
   'status_code': 404}},
 {'Input': {'szin': 'zold'},
  'output': 'ZOLD',
  'teamID': [],
  'channelID': {'id': 'api.context.404.app_error',
   'message': 'Sorry, we could not find the page.',
   'detailed_error': "There doesn't appear to be an api call for the url='/api/v4/teams/[]/channels'.",
   'status_code': 404}}]

TypeError: test_save() takes 0 positional arguments but 1 was given

#### Mattermost fuzzing

In [14]:
mr.create_a_post(channel_id=channel_id, message='this is a new message')

{'id': 'api.context.permissions.app_error',
 'message': 'You do not have the appropriate permissions',
 'detailed_error': '',
 'request_id': 'rsdhunhjpfbk5mgmd3ztx5ax3w',
 'status_code': 403}

In [15]:
mr.get_methods()

[('activate_plugin', ['plugin_id']),
 ('add_log_message', ['body']),
 ('add_multiple_users_to_team', ['team_id', 'body']),
 ('add_user_to_channel', ['channel_id', 'body']),
 ('add_user_to_team', ['team_id', 'body']),
 ('add_user_to_team_from_invite', ['hash', 'data', 'invite_id']),
 ('attach_mobile_device', ['body']),
 ('autocomplete_channels', ['team_id', 'name']),
 ('autocomplete_custom_emoji', ['name']),
 ('autocomplete_users', ['team_id', 'channel_id', 'name']),
 ('cancel_a_job', ['job_id']),
 ('check_if_team_exists', ['name']),
 ('check_mfa', ['body']),
 ('check_system_health', []),
 ('create_a_channel', ['body']),
 ('create_a_command', ['body']),
 ('create_a_custom_emoji', ['image', 'emoji']),
 ('create_a_direct_message_channel', ['body']),
 ('create_a_group_message_channel', ['body']),
 ('create_a_new_job', ['body']),
 ('create_a_post', ['post']),
 ('create_a_team', ['body']),
 ('create_a_user', ['body']),
 ('create_a_user_access_token', ['user_id', 'token']),
 ('create_an_incom

In [16]:
mr.get_teams()

[]

In [26]:
def test_save(testlist):
    f = open("archive/test-"+datetime.datetime.now().strftime("%y-%m-%d-%H-%M")+".txt", "w")
    for item in testlist:
        f.write("%s\n" % item)
    f.close()
test_save(apitestlist)